In [1]:
import sys
print(sys.executable)

/home/rs/PycharmProjects/snake-multi/.venv/bin/python3.10


In [2]:
import os
os.chdir("../..")
os.getcwd()

'/home/rs/PycharmProjects/snake-multi'

In [3]:
import random
import logging
import sys
from collections import deque
from functools import partial
from math import floor, ceil

import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from src.simulation.simulation import Simulation, Vector2
from src.simulation.board_generator import make_simple_board

from src.model.model_utils import get_model_moves, get_model_scores, get_rotated_state

tf.config.experimental.set_memory_growth(tf.config.list_physical_devices("GPU")[0], True)

2025-09-03 20:46:06.658076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756925166.669360   99096 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756925166.672889   99096 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756925166.682903   99096 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756925166.682915   99096 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756925166.682917   99096 computation_placer.cc:177] computation placer alr

In [4]:
view_range = 7
replay_buffer_size = 50_000

In [5]:
def calculate_score(ate_food: bool, died: bool):
    if died:
        return -10

    if ate_food:
        return 10

    return 0

In [6]:
cat_encoding = tf.keras.layers.CategoryEncoding(num_tokens=6, output_mode="one_hot")

cat_encoding([[3, 2, 1, 5], [5, 3, 1, 3]])

I0000 00:00:1756925168.438366   99096 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1574 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 3GB, pci bus id: 0000:01:00.0, compute capability: 6.1


<tf.Tensor: shape=(2, 4, 6), dtype=float32, numpy=
array([[[0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.]]], dtype=float32)>

In [7]:
#online_model = tf.keras.Sequential([
#    tf.keras.layers.Input((view_range * 2 + 1, view_range * 2 + 1)),
#    tf.keras.layers.Flatten(),
#    tf.keras.layers.CategoryEncoding(num_tokens=6, output_mode="one_hot"),
#    tf.keras.layers.Flatten(),
#    tf.keras.layers.Dense(42, activation="relu", kernel_initializer="he_normal"),
#    tf.keras.layers.Dense(42, activation="relu", kernel_initializer="he_normal"),
#    tf.keras.layers.Dense(1)
#])

DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same", activation="relu", kernel_initializer="he_normal")

online_model = tf.keras.Sequential([
    tf.keras.layers.Input((view_range * 2 + 1, view_range * 2 + 1)),
    tf.keras.layers.CategoryEncoding(num_tokens=7, output_mode="one_hot"),
    DefaultConv2D(filters=32, kernel_size=7),
    DefaultConv2D(filters=64, kernel_size=5),
    DefaultConv2D(filters=64, kernel_size=3),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1)
])

target_model = tf.keras.models.clone_model(online_model)
target_model.set_weights(online_model.get_weights())

view_type = "full"

In [8]:
model_name = ""

In [9]:
logging.basicConfig(level=logging.DEBUG)

log_formatter = logging.Formatter("%(message)s")

std_handler = logging.StreamHandler(sys.stdout)
std_handler.setLevel(logging.INFO)
std_handler.setFormatter(log_formatter)

file_handler = logging.FileHandler(f"logs_{model_name}.log")
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(log_formatter)

logger = logging.getLogger()
logger.handlers.clear()
logger.addHandler(std_handler)
logger.addHandler(file_handler)

In [10]:
online_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ category_encoding_1             │ (None, 15, 15, 7)      │             0 │
│ (CategoryEncoding)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 15, 15, 32)     │        11,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 15, 15, 32)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 7, 7, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,617 (436.00 KB)

 Trainable params: 111,617 (436.00 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
simulation = Simulation(calculate_score)
simulation.reset(make_simple_board(np.array([view_range * 2 + 1, view_range * 2 + 1])), 2, 1)

online_model.predict(np.array([simulation.get_snake_view(0, view_type, view_range)]))

I0000 00:00:1756925169.048954   99163 service.cc:152] XLA service 0x7f0b040047d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756925169.048971   99163 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 3GB, Compute Capability 6.1
2025-09-03 20:46:09.055482: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1756925169.080317   99163 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-09-03 20:46:09.264972: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.13 = (f32[1,32,15,15]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,32,15,15]{3,2,1,0} %bitcast.302, f32[32,32,5,5]{3,2,1,0} %bitcast.309, f32[32]{0} %bitcast.311), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudn

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 593ms/step


I0000 00:00:1756925169.578140   99163 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


array([[-0.215972]], dtype=float32)

In [12]:
def epsilon_greedy(snakes_states: list[np.ndarray], snakes_possible_moves: list[list[Vector2]], snakes_alive: list[bool], epsilon: float=0.):
    n_snakes = len(snakes_states)

    process_mask = [True for _ in range(n_snakes)]
    
    for i, alive in enumerate(snakes_alive):
        if not alive or np.random.rand() < epsilon:
            process_mask[i] = False

    moves = get_model_moves(online_model, snakes_states, snakes_possible_moves, process_mask, softmax=True)

    for i, move in enumerate(moves):
        if move == (0, 0):
            moves[i] = random.choice(simulation.get_legal_moves(i))

    return moves

In [13]:
class ReplayBuffer:
    def __init__(
        self,
        priority_size: int,
        normal_size: int,
        default_split: float = 0.2,
        default_batch_size: int = 32,
        threshold_decay: float = 0.3,
        threshold_multiplier: float = 1.3
    ):
        self.priority_buffer = deque(maxlen=priority_size)
        self.normal_buffer = deque(maxlen=normal_size)

        self.default_split = default_split

        self.loss_threshold = 0
        self.threshold_decay = threshold_decay
        self.threshold_multiplier = threshold_multiplier

        self.last_priority_idxs = []
        self.last_normal_idxs = []

    def append(self, experience: tuple):
        self.normal_buffer.append(experience)

    def sample(self, split: float = -1, batch_size: int = -1):
        if split < 1:
            split = self.default_split

        if batch_size < 1:
            batch_size = self.default_batch_size

        if len(self.priority_buffer) < batch_size * split:
            split = 0

        n_priority = ceil(split * batch_size)
        n_normal = floor((1 - split) * batch_size)

        batch = []

        if n_priority:
            idxs = np.random.randint(len(self.priority_buffer), size=n_priority)
            batch += [self.priority_buffer[i] for i in idxs]
            self.last_priority_idxs = idxs
        else:
            self.last_priority_idxs = []

        if n_normal:
            idxs = np.random.randint(len(self.normal_buffer), size=n_normal)
            batch += [self.normal_buffer[i] for i in idxs]
            self.last_normal_idxs = idxs
        else:
            self.last_normal_idxs = []
            
        return [
            [experience[field_idx] for experience in batch]
            for field_idx in range(6)
        ]

    def update_loss(self, losses: tf.Tensor, mean_loss: tf.Tensor):
        self.loss_threshold *= 1 - self.threshold_decay
        self.loss_threshold += self.threshold_decay * mean_loss

        for i, priority_idx in enumerate(self.last_priority_idxs):
            if losses[i] < self.loss_threshold * self.threshold_multiplier:
                if priority_idx != -1:
                    del self.priority_buffer[priority_idx]

                self.last_priority_idxs[self.last_priority_idxs == priority_idx] = -1
                self.last_priority_idxs[self.last_priority_idxs > priority_idx] -= 1

        for i, normal_idx in enumerate(self.last_normal_idxs, start=len(self.last_priority_idxs)):
            if losses[i] > self.loss_threshold * self.threshold_multiplier:
                self.priority_buffer.append(self.normal_buffer[normal_idx])

        logger.info(f"\tPriority: {len(self.priority_buffer)}")

    def clear(self):
        self.priority_buffer.clear()
        self.normal_buffer.clear()

        self.loss_threshold = 0

        self.last_normal_idxs = []
        self.last_priority_idxs = []

In [14]:
#priority_replay_buffer = deque(maxlen=replay_buffer_size // 2)
#replay_buffer = deque(maxlen=replay_buffer_size // 2)

replay_buffer = ReplayBuffer(replay_buffer_size // 2, replay_buffer_size // 2)

In [15]:
def play_one_step(simulation: Simulation, states_before: list[np.ndarray], possible_moves_before: list[Vector2], snakes_alive_before: list[bool], epsilon: float):
    # Save states
    moves = epsilon_greedy(states_before, possible_moves_before, snakes_alive_before, epsilon)

    scores, running = simulation.next(moves)

    states_after = [simulation.get_snake_view(i, view_type, view_range) for i in range(simulation.n_snakes)]
    possible_moves_after = [simulation.get_legal_moves(i) for i in range(simulation.n_snakes)]
    snakes_alive_after = simulation.snakes_alive

    for i in range(len(states_before)):
        if snakes_alive_before[i]:
            replay_buffer.append((states_before[i], moves[i], scores[i], states_after[i], possible_moves_after[i], snakes_alive_after[i]))

    return states_after, possible_moves_after, snakes_alive_after, scores, running

In [16]:
def sample_experiences(batch_size: int):
    idxs = np.random.randint(len(replay_buffer), size=batch_size)
    
    batch = [replay_buffer[idx] for idx in idxs]

    return [
        [experience[field_idx] for experience in batch]
        for field_idx in range(6)
    ]

In [17]:
batch_size = 32
discount_factor = 0.9
loss_fn = tf.keras.losses.mse
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001)

losses = []


def training_step(buffer_split: float):
    #experiences = sample_experiences(batch_size)
    experiences = replay_buffer.sample(split=buffer_split, batch_size=batch_size)
    states, moves, scores, states_after, possible_moves_after, snakes_alive_after = experiences

    best_next_moves = [[move] for move in get_model_moves(online_model, states_after, possible_moves_after, snakes_alive_after)]
    next_q_values = get_model_scores(target_model, states_after, best_next_moves, snakes_alive_after)
    
    #next_q_values = get_model_scores(target_model, states_after, possible_moves_after, snakes_alive_after)
    runs = 1.0 - np.array(snakes_alive_after)
    
    target_q_values = scores + runs * discount_factor * next_q_values

    X = tf.constant([get_rotated_state(state, move) for state, move in zip(states, moves)])
    
    with tf.GradientTape() as tape:
        predicted_q_values = online_model(X)
        individual_losses = loss_fn(target_q_values, predicted_q_values)
        loss = tf.reduce_mean(individual_losses)

    replay_buffer.update_loss(individual_losses, loss)

    losses.append(loss.numpy())
    grads = tape.gradient(loss, online_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, online_model.trainable_variables))

In [18]:
simulation = Simulation(calculate_score)
rewards = []
steps = []
replay_buffer.clear()

best_model_window = 7
best_model_rewards = float("-inf")

n_episodes = 100_000

for episode in range(n_episodes):
    simulation.reset(make_simple_board(np.array([15, 15])), snake_count=1, food_count=1)

    rewards.append(0)

    states = [simulation.get_snake_view(i, view_type, view_range) for i in range(simulation.n_snakes)]
    possible_moves = [simulation.get_legal_moves(i) for i in range(simulation.n_snakes)]
    snakes_alive = simulation.snakes_alive
    
    epsilon = max(1 - (episode / (n_episodes * 0.8)), 0.005)
    priority_buffer_split = min(episode / (n_episodes * 0.8) * 0.2, 0.2)

    for step in range(200):
        #discount_factor = 0.95 * (episode / n_episodes)
        
        states, possible_moves, snakes_alive, scores, running = play_one_step(simulation, states, possible_moves, snakes_alive, epsilon)

        rewards[-1] += sum(scores)

        if not running:
            break

    steps.append(step)

    logging.info(f"Episode {episode} played")
    logging.info(f"\tSteps: {step}")
    logging.info(f"\tRewards: {rewards[-1]}")

    if episode > 200:
        curr_model_rewards = sum(rewards[-best_model_window:])
        if curr_model_rewards > best_model_rewards:
            avg_rewards = curr_model_rewards / best_model_window
            best_model_rewards = curr_model_rewards
            logging.info(f"\tSnaphshot saved (avg. {avg_rewards})")
            
            if model_name:
                online_model.save(f"models/{model_name}_{round(avg_rewards)}_snapshot.keras")

        training_step(priority_buffer_split)
        logging.info("\tTraining finished")
        
        if episode % 100 == 0:
            target_model.set_weights(online_model.get_weights())
            logging.info("\tTarget model updated")

Episode 0 played
	Steps: 20
	Rewards: -10
Episode 1 played
	Steps: 1
	Rewards: -10
Episode 2 played
	Steps: 31
	Rewards: 0
Episode 3 played
	Steps: 7
	Rewards: -10
Episode 4 played
	Steps: 12
	Rewards: -10
Episode 5 played
	Steps: 40
	Rewards: 0
Episode 6 played
	Steps: 22
	Rewards: -10
Episode 7 played
	Steps: 51
	Rewards: -10
Episode 8 played
	Steps: 19
	Rewards: -10
Episode 9 played
	Steps: 72
	Rewards: 0
Episode 10 played
	Steps: 11
	Rewards: -10
Episode 11 played
	Steps: 43
	Rewards: -10
Episode 12 played
	Steps: 3
	Rewards: -10
Episode 13 played
	Steps: 84
	Rewards: -10
Episode 14 played
	Steps: 14
	Rewards: -10
Episode 15 played
	Steps: 3
	Rewards: -10
Episode 16 played
	Steps: 2
	Rewards: -10
Episode 17 played
	Steps: 34
	Rewards: -10
Episode 18 played
	Steps: 1
	Rewards: -10
Episode 19 played
	Steps: 8
	Rewards: -10
Episode 20 played
	Steps: 18
	Rewards: -10
Episode 21 played
	Steps: 9
	Rewards: -10
Episode 22 played
	Steps: 36
	Rewards: -10
Episode 23 played
	Steps: 2
	Reward

2025-09-03 20:46:09.962696: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.13 = (f32[3,32,15,15]{3,2,1,0}, u8[0]{0}) custom-call(f32[3,32,15,15]{3,2,1,0} %bitcast.289, f32[32,32,5,5]{3,2,1,0} %bitcast.296, f32[32]{0} %bitcast.298), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1_2/convolution" source_file="/home/rs/PycharmProjects/snake-multi/.venv/lib64/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-09-03 20:46:10.035692: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:54

Episode 66 played
	Steps: 34
	Rewards: -10
Episode 67 played
	Steps: 6
	Rewards: -10
Episode 68 played
	Steps: 3
	Rewards: -10
Episode 69 played
	Steps: 48
	Rewards: -10
Episode 70 played
	Steps: 1
	Rewards: -10
Episode 71 played
	Steps: 0
	Rewards: -10
Episode 72 played
	Steps: 1
	Rewards: -10
Episode 73 played
	Steps: 84
	Rewards: -10
Episode 74 played
	Steps: 1
	Rewards: -10
Episode 75 played
	Steps: 9
	Rewards: -10
Episode 76 played
	Steps: 2
	Rewards: -10
Episode 77 played
	Steps: 5
	Rewards: -10
Episode 78 played
	Steps: 53
	Rewards: -10
Episode 79 played
	Steps: 1
	Rewards: -10
Episode 80 played
	Steps: 4
	Rewards: -10
Episode 81 played
	Steps: 11
	Rewards: -10
Episode 82 played
	Steps: 14
	Rewards: -10
Episode 83 played
	Steps: 1
	Rewards: -10
Episode 84 played
	Steps: 14
	Rewards: -10
Episode 85 played
	Steps: 2
	Rewards: -10
Episode 86 played
	Steps: 17
	Rewards: 0
Episode 87 played
	Steps: 5
	Rewards: -10
Episode 88 played
	Steps: 13
	Rewards: -10
Episode 89 played
	Steps: 1

2025-09-03 20:46:10.833038: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.13 = (f32[4,32,15,15]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,32,15,15]{3,2,1,0} %bitcast.289, f32[32,32,5,5]{3,2,1,0} %bitcast.296, f32[32]{0} %bitcast.298), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1_2/convolution" source_file="/home/rs/PycharmProjects/snake-multi/.venv/lib64/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-09-03 20:46:10.889678: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:54

Episode 184 played
	Steps: 5
	Rewards: -10
Episode 185 played
	Steps: 34
	Rewards: -10
Episode 186 played
	Steps: 5
	Rewards: -10
Episode 187 played
	Steps: 39
	Rewards: -10
Episode 188 played
	Steps: 8
	Rewards: -10
Episode 189 played
	Steps: 15
	Rewards: -10
Episode 190 played
	Steps: 2
	Rewards: -10
Episode 191 played
	Steps: 15
	Rewards: 0
Episode 192 played
	Steps: 1
	Rewards: 0
Episode 193 played
	Steps: 1
	Rewards: -10
Episode 194 played
	Steps: 0
	Rewards: -10
Episode 195 played
	Steps: 0
	Rewards: -10
Episode 196 played
	Steps: 0
	Rewards: -10
Episode 197 played
	Steps: 0
	Rewards: -10
Episode 198 played
	Steps: 11
	Rewards: -10
Episode 199 played
	Steps: 8
	Rewards: -10
Episode 200 played
	Steps: 32
	Rewards: -10
Episode 201 played
	Steps: 50
	Rewards: -10
	Snaphshot saved (avg. -10.0)


...

2025-09-03 20:46:19.138344: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.13 = (f32[11,32,15,15]{3,2,1,0}, u8[0]{0}) custom-call(f32[11,32,15,15]{3,2,1,0} %bitcast.289, f32[32,32,5,5]{3,2,1,0} %bitcast.296, f32[32]{0} %bitcast.298), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1_2/convolution" source_file="/home/rs/PycharmProjects/snake-multi/.venv/lib64/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-09-03 20:46:19.180292: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:

	Priority: 17
	Training finished
Episode 212 played
	Steps: 32
	Rewards: -10
	Priority: 18
	Training finished
Episode 213 played
	Steps: 2
	Rewards: -10


2025-09-03 20:46:20.329113: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.13 = (f32[28,32,15,15]{3,2,1,0}, u8[0]{0}) custom-call(f32[28,32,15,15]{3,2,1,0} %bitcast.289, f32[32,32,5,5]{3,2,1,0} %bitcast.296, f32[32]{0} %bitcast.298), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1_2/convolution" source_file="/home/rs/PycharmProjects/snake-multi/.venv/lib64/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-09-03 20:46:20.410301: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:

	Priority: 23
	Training finished
Episode 214 played
	Steps: 12
	Rewards: -10
	Priority: 16
	Training finished
Episode 215 played
	Steps: 0
	Rewards: -10
	Priority: 16
	Training finished
Episode 216 played
	Steps: 19
	Rewards: -10
	Priority: 16
	Training finished
Episode 217 played
	Steps: 40
	Rewards: -10
	Priority: 17
	Training finished
Episode 218 played
	Steps: 1
	Rewards: -10
	Priority: 20
	Training finished
Episode 219 played
	Steps: 32
	Rewards: -10
	Priority: 21
	Training finished
Episode 220 played
	Steps: 10
	Rewards: -10
	Priority: 23
	Training finished
Episode 221 played
	Steps: 1
	Rewards: -10
	Priority: 24
	Training finished
Episode 222 played
	Steps: 27
	Rewards: -10
	Priority: 26
	Training finished
Episode 223 played
	Steps: 12
	Rewards: -10
	Priority: 28
	Training finished
Episode 224 played
	Steps: 4
	Rewards: -10
	Priority: 30
	Training finished
Episode 225 played
	Steps: 5
	Rewards: -10
	Priority: 30
	Training finished
Episode 226 played
	Steps: 30
	Rewards: -10
	Pri

2025-09-03 20:46:24.473316: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.13 = (f32[21,32,15,15]{3,2,1,0}, u8[0]{0}) custom-call(f32[21,32,15,15]{3,2,1,0} %bitcast.289, f32[32,32,5,5]{3,2,1,0} %bitcast.296, f32[32]{0} %bitcast.298), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1_2/convolution" source_file="/home/rs/PycharmProjects/snake-multi/.venv/lib64/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-09-03 20:46:24.526984: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:

In [ ]:
#sns.lineplot(steps, label="steps")
fig, ax = plt.subplots(figsize=(16,6))
sns.lineplot(np.convolve(rewards, [1 for _ in range(10)]))
plt.xlabel("Episode")
plt.ylabel("Reward")

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
sns.lineplot(np.convolve(losses, [1 for _ in range(10)]), ax=ax)
plt.xlabel("Episode")
plt.ylabel("Loss")

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
sns.lineplot(np.cumsum(rewards), label="rewards", ax=ax)
plt.xlabel("Episode")
plt.ylabel("Reward")

In [ ]:
online_model.save(f"models/{model_name}.keras")